# Encripción Completamente Homomorfa con Polinomios sobre el Toro


**Preliminares:**

Usaremos las siguientes definiciones en el resto del proyecto:
- $\mathbb{R}$ y $\mathbb{Z}$ denotan los $\mathbb{Z}$-módulos de los reales y los enteros respectivamente
- $ \mathbb{T} = \mathbb{R} / \mathbb{Z}$, los reales módulo 1, denota el toro unidimensional (el círculo) como $\mathbb{Z}$-módulo
- Sean $N$ una potencia de $2$ dada. $\Phi(x) = x^N +1$, irreducible sobre $\mathbb{Z}$
- Denotamos los cocientes $\mathbb{R}_N [x] = \mathbb{R}[x] / \Phi(x), \mathbb{Z}_N [x] = \mathbb{Z}[x] / \Phi(x)$ y $\mathbb{T}_N [x] = \mathbb{T}[x] / \Phi(x) = \mathbb{R}_N [x] / \mathbb{Z}_N [x] $ vistos como $\mathbb{Z}_N [x]$-módulos
- $\mathbb{B} = \{0,1 \}$

El sistema de encripción que usamos se basa en el siguiente problema:

**LWE Sobre el Toro**
Dados $n \in \mathbb{N}$, $\chi$ una distribución sobre $\mathbb{R}$, y $\mathbf{s} =(s_1,...,s_n)  \xleftarrow{\$} \mathbb{B}^n $, el problema es distinguir las siiguientes distribuciones:
- $\mathcal{D}_0 = \{ (\mathbf{a},r) : \mathbf{a} \xleftarrow{\$} \mathbb{T}^n, r \xleftarrow{\$}\mathbb{T}  \}$
- $\mathcal{D}_1 = \{ (\mathbf{a},r) : \mathbf{a} = (a_1,...,a_n) \xleftarrow{\$} \mathbb{T} ^n, r = \sum_{i=1}^{n} a_i \cdot s_i + e  \}$, con $e \xleftarrow{\chi} \mathbb{R}$

Básicamente, es difícil distinguir elementos $r \in \mathbb{T}$ verdaderamente aleatorios, de elementos construidos como cierta combinación lineal de otros elementos del toro (las componentes de $\mathbf{a}$) junto con cierto residuo $e$, incluso teniendo acceso a $\mathbf{a}$

<img src="misc/1.jpg" width="500" />

Ahora, como esto se va a trabajar dentro de un computador, necesitamos  cierta precisión decimal, que va a ser fija. Si tenemos $\Omega$ bits de precisión, esencialmente estamos reemplazando a $\mathbb{T}$ por el siguiente módulo:
$$
\hat{\mathbb{T}} = \{ t = \sum_{i=0}^\Omega 2^{-i} \cdot t_i \quad \text{mod} \quad 1 | t_i \in \mathbb{B}  \} = 2^{-\Omega} \mathbb{Z} / \mathbb{Z}   
$$
Llamemos $q = 2^{-\Omega}$. Podemos tomar representantes $\hat{\mu} \in \mathbb{Z}_q$ de elementos de $\mu \in \hat{ \mathbb{T}}$, tales que $\mu = q^{-1} \hat{\mu} \quad mod \quad 1$. Por ejemplo, usando $\Omega = 4$, tenemos lo siguiente:
$$
\begin{align*}
    &q = 16 \\
    &\hat{\mathbb{T}} = \frac{1}{16} \mathbb{Z} / \mathbb{Z} &&= \left\{ \mu =  \frac{t_1}{2} + \frac{t_2}{4} + \frac{t_3}{8} + \frac{t_4}{16} \quad \text{mod} \quad : t_i \in \mathbb{B} \right\}\\
   & &&= \left\{ \mu = \frac{1}{16}[ 8t_1 + 4t_2 + 2t_3 + t_4] \quad \text{mod} \quad : t_i \in \mathbb{B} \right\{ \\
   & &&= \left\{ \mu = \frac{1}{16} \hat{\mu} : \hat{\mu} \in \mathbb{Z}_{16} \right\}
\end{align*}
$$
Y podemos entonces codificar en total $q = 16$ números. Usualmente se toma $\Omega = 32$ o $\Omega = 64$.

Para poder añadir espacio donde acomodar el ruido que esconderá los textos, e implementar ciertas operaciones homomorfas, necesitamos establecer ciertos bits significativos $ \hat{\omega} $ a cero, y almacenamos realmente usando $\omega$ bits de precisión, donde $\hat{\omega} + \omega \leq \Omega $. Con esto obtenemos la asiguiente representación para elementos del toro:

<img src="misc/2.jpg" width="500" />

LLamemos $P = 2^{\omega + \hat{\omega}}$, que corresponde a la cantidad de $\hat{X_H}$ posibles, y representamos $\hat{\mu}$ como $\hat{\mu} = 2 ^{\Omega - (\omega + \hat{\omega})} \cdot ( \hat{v} \quad mod \quad 2^\omega) $
Si $\hat x \in \mathbb{Z}_q$, entonces $\hat{x}  = \frac{q}{p} \hat{x_H} + \hat{x_L}, \quad \frac{-q}{2p} \leq \hat{x_L} \leq \frac{q}{2p}$. $\hat{X_L}$ se comporta como el ruido de las encripciones, por lo que interesa definir $Upper(\frac{q}{p} \hat{x_H }+ \hat{x_L}) = \frac{q}{p} \hat{x_H} + 0$ para reiniciar el ruido. En términos explícitos, 
$$
Upper(\hat{x}) = \frac{q}{p} \left \lfloor \frac{p}{q} lift(\hat{x}) \right\rceil \quad mod \quad q 
$$

Esto nos permite tener elementos esencialmente iguales, si coinciden bajo $Upper$, y realmente trabajamos sobre la siguiente estructura:

<img src="misc/3.jpg" width="500" />

Con esto, tenemos todo lo que necesitamos para introducir el sistema de encripción: 
1. $\text{KeyGen}(1^\lambda)$: En un parámetro de seguridad $\lambda$:
    1. Fijamos $n \leftarrow n(\lambda)$, una distribución de error $\chi = \mathcal{N}(0,\sigma^2)$
    2. Generamos la llave secreta $ \mathcal{sk} = \vec{s} = (s_1, ... , s_n) \xleftarrow{ \$ } \mathbb{B}^n$
    - Recibimos textos planos $ \hat{\mu} \in \mathcal{P} := \frac{p}{q} \ \mathbb{Z} / q \mathbb{Z}$, con $p = 2^{\omega + \hat{\omega}}$ el módulo/parte alta del mensaje, y $q = 2^\Omega$.
    - Los parámetros $ \{ n, \sigma, p, q \}$ son todos públicos.
2. $\text{Encrypt}_{\mathcal{sk}}(\hat{\mu})$: Recibimos un texto plano $\hat{\mu} \in \mathcal{P}$:
    1. Sampleamos $\vec{a} = (\hat{a_1},...,\hat{a_n}) \xleftarrow{ \$ } \mathbb{Z} \ / q\mathbb{Z} ^n$, $e \xleftarrow{\chi} \mathbb{R}$, y  $\hat{e} = \left \lfloor e q \right \rceil$
    2. Hacemos $\hat{\mu} ^* = \hat{\mu} + \hat{e}$: Introducimos error/ruido al texto plano
    3. Hacemos $ \hat{b} = \sum_{i=1}^n s_i \hat{a_i} + \hat{\mu} ^* \quad mod \quad q$
    4. Devolvemos $ \hat{c} = \overline{LWE}_{\mathcal{sk}}(\hat{\mu}) = (\hat{a_1} , ... , \hat{a_n} , \hat{\mathcal{b}})$
3. $\text{Decrypt}_{\mathcal{sk}} (\hat{\mathcal{c}} )$. En posesión de la llave $\mathcal{sk} = \vec{s}$, calculamos
    1. $\hat{\mu}^* = \hat{\mathcal{b}} - \sum _{i=1} ^{n} s_i \hat{a_i} $
    2. Eliminamos el ruido con $\text{Upper}(\hat{\mu}^*)$
    Siempre que $||\hat{e}|| < \frac{q}{2p}, \text{Upper}(\hat{\mu}^*) = \hat{\mu}$ y la desencripción es correcta

<img src="misc/4.jpg" width="450" /> <img src="misc/5.jpg" width="500" />


Ahora bien, este modelo de encripción se generaliza inmediatamente a polinomios sobre el toro, reemplazando todas las operaciones módulo q por operaciones módulo q y módulo $x^N +1$ respectivamente. Quedamos con el siguiente algoritmo de encripción:
1. $\text{KeyGen}(1^\lambda)$: En un parámetro de seguridad $\lambda$:
    1. Fijamos $k,N \leftarrow n(\lambda)$, una distribución de error $\chi = \mathcal{N}(0,\sigma^2)$
    2. Generamos la llave secreta $ \mathcal{sk} = \vec{\mathcal{s}} = (\mathcal{s}_1, ... , \mathcal{s}_n) \xleftarrow{ \$ } \mathbb{B}_N[x] ^k$
    - Recibimos textos planos $ \hat{\mu} \in \mathcal{P}_N := \frac{p}{q} \ \mathbb{Z} / q \mathbb{Z} [x] / \langle x^N +1 \rangle =: \hat{\mathbb{Z}}_N$, con $p = 2^{\omega + \hat{\omega}}$ el módulo/parte alta del mensaje, y $q = 2^\Omega$.
    - Los parámetros $ \{ k,N, \sigma, p, q \}$ son todos públicos.
2. $\text{Encrypt}_{\mathcal{sk}}(\hat{\mu})$: Recibimos un texto plano $\hat{\mu} \in \mathcal{P}_N$:
    1. Sampleamos $\vec{\mathcal{a}} = (\hat{\mathcal{a}_1},...,\hat{\mathcal{a}_n}) \xleftarrow{ \$ } \hat{\mathbb{Z}}_N $ , $e \xleftarrow{\chi} \mathbb{R}_N[x]$, y  $\hat{e} = \left \lfloor e q \right \rceil$
    2. Hacemos $\hat{\mu} ^* = \hat{\mu} + \hat{e}$
    3. Hacemos $ \hat{\mathcal{b}} = \sum_{i=1}^n s_i \hat{a_i} + \hat{\mu} ^* \quad mod \quad (q, x^N +1)$
    4. Devolvemos $ \hat{c} = \overline{GLWE}_{\mathcal{sk}}(\hat{\mu}) = (\hat{a_1} , ... , \hat{a_n} , \hat{\mathcal{b}}) \in \hat{\mathbb{Z}}_N ^k$
3. $\text{Decrypt}_{\mathcal{sk}} (\hat{\mathcal{c}} )$. En posesión de la llave $\mathcal{sk} = \vec{s}$, calculamos
    1. $\hat{\mu}^* = \hat{\mathcal{b}} - \sum _{i=1} ^{n} s_i \hat{\mathcal{a}_i} $
    2. Eliminamos el ruido con $\text{Upper}(\hat{\mu}^*)$
    Siempre que $||\hat{e}|| < \frac{q}{2p}, \text{Upper}(\hat{\mu}^*) = \hat{\mu}$ y la desencripción es correcta

## Operaciones "niveladas"/ SHE

Sean $\hat{\mu_1},\hat{\mu_2} \in \mathcal{P}_N, \vec{s} \in \mathbb{B}_N[x] ^k$. Como encriptamos enviando elementos a un módulo, la encripción respeta sumas de manera prácticamente trivial. Sean
$$
\bar{c}_1 \leftarrow \overline{\text{GLWE}}_s(\bar{\mu}_1) \in \mathbb{Z}_N^k[x] \times \mathbb{Z}_N[x], \quad
\bar{c}_2 \leftarrow \overline{\text{GLWE}}_s(\bar{\mu}_2) \in \mathbb{Z}_N^k[x] \times \mathbb{Z}_N[x]
$$

Sabemos que:
$$
\bar{c}_1 = \langle \bar{s}, \bar{a}_1 \rangle + \bar{\mu}_1 + \bar{e}_1 , \quad
\bar{c}_2 = \langle \bar{s}, \bar{a}_2 \rangle + \bar{\mu}_2 + \bar{e}_2
$$

Luego:
$$
\begin{align*}
\bar{c}_1 + \bar{c}_2 &= \langle \bar{s}, \bar{a}_1 \rangle + \bar{\mu}_1 + \bar{e}_1 + \langle \bar{s}, \bar{a}_2 \rangle + \bar{\mu}_2 + \bar{e}_2 \\
&= \langle \bar{s}, \bar{a}_1 + \bar{a}_2 \rangle + \bar{\mu}_1 + \bar{\mu}_2 + \bar{e}_1 + \bar{e}_2
\end{align*}

$$

Por lo tanto, si $ || \bar{e_1} + \bar{e_2} ||_{\infty} < \frac{p}{2q}$, entonces:

$$
\text{Decrypt}_{\vec{s}}(\bar{c}_1 + \bar{c}_2) = \bar{\mu}_1 + \bar{\mu}_2 \quad \checkmark
$$

Esto inmediatamente define un producto por escalares en $ \mathbb{Z} $
$$
k \cdot \bar{c} =
\begin{cases}
\bar{c} + \dots + \bar{c} & \quad k > 0 \\
(-\bar{c}) + \dots + (-\bar{c}) & \quad k \leq 0
\end{cases}
$$
Y esto también generaliza a productos por polinomios en $\mathbb{Z}_N[X]$, exandiendo componente a componente, siempre que el ruido se mantenga pequeño. Sin embargo, aqui hay un detalle muy importante, y es que el escalar debe estar en claro, lo cual no es necesariamente deseable. Adempas, no podemos implementar realmente un producto entre dos textos cifrados todavía, y falta para que este sistema pueda ser "bootstrappeable"

Definimos un modelo de encripción auxiliar: $\text{GGSW}$

Usamos la misma llave $\mathcal{sk} = \vec{\mathcal{s}} = (\mathcal{s}_1, ... , \mathcal{s}_n) \xleftarrow{ \$ } \mathbb{B}_N[x] ^k $ que en $\text{GLWE}$, pero necesitamos parámetros adicionales $B = 2^\beta, l$, con $\beta, l \geq 1$ y $\beta \cdot l \leq \Omega$.
Definimos
$
\vec{g} = (2^{2-\beta}, 2^{2-2\beta}, \dots, 2^{2-l\beta})
$

Encriptamos textos planos $m \in \mathbb{Z}_N[x]$ como sigue:

$$
\tilde{c} \leftarrow \text{GGSW}_s(m) = \overline{\mathcal{L}} + m G^T \in \mathbb{Z}_N[x]^{(l+1) \times (l+1)}
$$
$\overline{\mathcal{L}}$ Corresponde a una matriz con $l(k+1)$  encripciones del 0, y $G^T$ una matriz $l(k+1) \times (k+1)$ con entradas en $\mathbb{Z} / q\mathbb{Z}$
$$
\overline{\mathcal{L}} =
\begin{bmatrix}
\text{GLWE}_s(0) \\
\vdots \\
\text{GLWE}_s(0)
\end{bmatrix}, \quad

G^T = \left[\begin{array}{c|c|c|c}
 \begin{matrix}\uparrow \\ \vec{g}^T \\ \downarrow\end{matrix} & 0 &...   & 0 \\
\hline
0 & \begin{matrix}\uparrow \\ \vec{g}^T \\ \downarrow\end{matrix} & ... & 0 \\
\hline
\vdots & & &\vdots \\
\hline
0 & 0 & ... & \begin{matrix}\uparrow \\ \vec{g}^T \\ \downarrow\end{matrix}
\end{array}\right]
$$

A la matriz $G$ se le llama matriz e descomposición "gadget", pues ayuda a descomponer elementos en $\mathbb{Z}_q$ como combinaciones lineal de elementos en $\vec{g}$, como si fueran dígitos:\
Si $d \in \mathbb{Z}/q\hat{\mathbb{Z}} = [ -q/2, q/2 )$, entonces $d \approx q \sum_{i=1}^l d_i \beta^i = q \sum_{i=1}^l d_i 2^{-i \beta} = \sum_{i=1}^l d_i 2^{\Omega-i \beta} = \langle g^{-1}(d) g \rangle = g^{-1}(d) \cdot g^T$, donde $g^{-1}(d) = (d_1,...,d_2)$ es el vector con los dígitos de la aproximación, que además cumple que $| g^{-1}(d) \cdot g^T - d| < 2^{\Omega - \beta l}/2$, que permite ajustar la precisión deseada modificando los parámetros.\
Esta descomposición se extiende inmediatamente a:

1. Polinomios: Tenemos
$
   \mathcal{P} = p_0 + p_1 x + \dots + p_N x^N \in \hat{\mathbb{Z}}_N[x]
$

Haciendo $g^{-1} (\mathcal{p}) = g^{-1} (p_0) , \dots, g^{-1}(p_{N-1})x^{N-1} \in \hat{\mathbb{Z}}_N^{l}[x]$, descomponiendo coeficiente a coeficiente obtenemos una aproximación $p \approx g^{-1}(p) \dot g^T$, con $|| g^{-1}(\mathcal{P}) g^T - \mathcal{P} || < 2^{\Omega-l \beta} / 2$

2. Vectores de polinomios. Trabajamos ahora con
$
   \vec{\mathcal{P}} = \begin{pmatrix} \mathcal{P}_1 \\ \vdots \\ \mathcal{P} _{k+1} \end{pmatrix} \in {\hat{\mathbb{Z}}_N}[x]^{l(k+1)}
$

Hacemos la descomposición componente a componente y obtenemos $G^{-1}(\ \vec{\mathcal{P}} ) =\begin{pmatrix} g^-1(\mathcal{P}_1) \\ \vdots \\ g^-1(\mathcal{P}_{k+1}) \end{pmatrix} \in \mathbb{Z}_N[x]^{l \times (k+1)}$\
Y también cumple $||G^{-1}(\vec{\mathcal{P}}) G^T - \vec{\mathcal{P}}  || \leq 2^{\Omega-l \beta} / 2$

Con toda esta maquinaria nueva, podemos definir un producto externo entre encripciones de GGSW y GLWE:


Sea $m_1\in \mathbb{Z}_N[x]$, $\overline{\mu_2} \in P_N[x]$:

$\overline{\mathcal{C}}_1 \leftarrow \text{GGSW}_s (m_1)$, y $\overline{c_2} \leftarrow \text{GLWE}_s (\overline{\mu_2})$

Definimos:

$$
\begin{align*}
\overline{\mathcal{C}}_1 \boxdot \overline{c}_2 &= G^{-1} (\overline{c}_2) \overline{\mathcal{C}}_1 \\
&= G^{-1} (\overline{c}_2)(\overline{\mathcal{L}} + m_1 G^T) \\
&= G^{-1}(\overline{c}_2)\overline{\mathcal{L}} + m_1 G^{-1} (\overline{c}_2) G^T \\
&= \begin{pmatrix}
    d_{00} + ... + d_{0(N-1)}x^{N-1}\\
    \vdots\\
    d_{(k+1)0} + ... + d_{(k+1)(N-1)}x^{N-1}
\end{pmatrix} ^T \begin{pmatrix}
    \overline{\text{GLWE}_{\vec{s}}}(0) \\
    \vdots\\
    \overline{\text{GLWE}_{\vec{s}}}(0)
\end{pmatrix} + m_1 G^{-1} (\overline{c}_2) G^T \\
&= \overline{\text{GLWE}_{\vec{s}}}(0) + m_1 G^{-1} (\overline{c}_2) G^T \\
&\approx \overline{\text{GLWE}_{\vec{s}}}(0) + m_1 \overline{c}_2 
\end{align*}
$$
Asi, si la descomposición gadget de $m_1$ es suficientemente buena, esto resulta ser una encripción válida de $m_1 \overline{\mu_2}$, y se tiene la ventaja de que $m_1$ se provee encriptado, en vez de antes que se tenia en plano. \
Con todo esto, podemos implementar la siguiente compuerta, que es de mucha utilidad no solo para construir circuitos nivelados, sino para hacer bootstrapping. Sean $\mathcal{C}_1 \leftarrow \text{GGSW}(b)$ para $ b \in \mathbb{B}$, y  $ \overline{c}_1, \overline{c}_0 \leftarrow \overline{\text{GLWE}}(\overline{\mu}_1, \overline{\mu}_2)$ para $\overline{\mu}_1, \overline{\mu}_2 \in \hat{\mathbb{Z}}_N [x]$

$$
\text{Cmux}(\mathcal{C}_1, \overline{c}_0, \overline{c}_1) = 
\begin{cases} 
\overline{c}_0 & \text{si } \mathcal{C}_1 =  \overline{\text{GGSW}}(0) \\
\overline{c}_1 & \text{si } \mathcal{C}_1=   \overline{\text{GGSW}}(1)
\end{cases} = \mathcal{C}_1 \boxdot ( \overline{c}_1 - \overline{c}_0) + \overline{c}_0
$$
Que corresponde a un condicional, bajo el bit $\mathcal{C}_1 $, que está encriptado!

Con esto, ya tenemos lo suficiente para armar el bootstrapping (programable)

### Bootstrapping

Tenemos un texto cifrado $\overline{c} \leftarrow \text{LWE}(\overline{\mu_1}) =  (\overline{a}_1, ... ,\overline{a}_k , b) \in \mathbb{Z}/q\mathbb{Z}^{n+1}$ bajo la llave secreta  $\vec{s} = (s_1,...,s_n) \in \mathbb{B}^n$\
Hay 2 pasos para desencriptar $\overline{c}$ y recuperar $\overline{\mu_1}$:
$$ \textbf{1)}\quad \overline{\mu_1}^* = b - \sum_{i=1}^n \overline{a}_i s_i \in \mathbb{Z} / q\mathbb{Z} \quad \quad \mapsto \quad \quad \textbf{2)} \quad \overline{\mu_1} = \overline{\mu_1}^* - e = \text{Upper}(\overline{\mu_1}^*) $$
Como en *1)* $\overline{\mu_1}^* \in \mathbb{Z} / q\mathbb{Z}$, existen $q$ valores posibles que puee tomar. Trataremos de construir entonces lo siguiente: un polinomio $\overline{v} = \overline{v}_0 + \overline{v} _1 x + ... + \overline{v}-{q-1} x^{q-1}, \overline{v} _i = \text{Upper}(i)$ llamado polinomio de prueba, tal que en sus coeficientes contenga versiones "sin ruido" de los exponentes.\
Si quisieramos entonces tomar un valor fresco para $\overline{\mu_1}^*$, podríamos hacer lo siguiente:

<img src="misc/6.jpg" width="500" />

$$ x^{-\overline{\mu_1}^*} \cdot \overline{v} = \overline{v}_i + \overline{v}_{i+1}x + ... + \overline{v}_{i-1} x^{q-1} $$
donde $\overline{v} = \overline{\mu_1} $ es el coeficiente "limpio" que le corresponde a $\overline{\mu_1}^*$!. Solo restaría ver como extraer el término constante de un polinomio, y con esto podriamos obtener una misma encripción para $\overline{\mu_1}$, y al ser el resultado justo después de un $\text{Upper}$, sabemos que tiene la menor cantidad de ruido posible. Ahora, como esto es bootstrapping, se debe hacer todo de manera homomorfa bajo unas llaves de bootstrapping, y a principio no es obvio como se puede implementar esto.\
Aprovecharemos la estructura polinomial de $x^{-\overline{\mu_1}^*} \cdot \overline{v}$ para trabajar dentro de encripciones de GLWE.


$x \in \mathbb{Z}_N[x]$ como elemento multiplicativo tiene orden $2N$, pero $\bar{\mu}^i$ está dado módulo $q$, por lo que debemos aproximarlo módulo $2N$ como:

$$
-\overline{\mu}^* \approx -\tilde{\mu}^* = -\tilde{b} + \sum_{i=1}^n s_i \tilde{a}_i \pmod{2N}, \quad \tilde{b} = \left\lfloor \frac{2N}{q} \overline{b} \quad mod \quad q \right\rceil \quad \tilde{a}_i = \left\lfloor \frac{2N}{q} \overline{a}_i \quad mod \quad q\right\rceil
$$

Con esto, en teoría hay $2N$ valores para $-\tilde{\mu}^*$, pero $\overline{v} \in \mathbb{Z}_N[x]$, solo tiene $N$ coeficientes, y noo alcanza para codificar todos los valores posibles. Para esto,  establecemos siempre el primer bit más significativo de $-\tilde{\mu}^*$ como $0$ (osea, establecer el parámetro $\overline{omega} \geq 1$ que se uso en la definición de partes altas y bajas de textos planos), lo que restringe a máximo $N$ valores posibles, y permite codificarlos todos en $\overline{v}$ asi´:
$$
\overline{v} = \overline{v}_0 + \overline{v}_1 x + \overline{v}_2 x^2 + \dots + \overline{v}_{N-1} x^{N-1} \in \hat{\mathbb{Z}}_{N}[x], \quad \overline{v}_i = \text{Upper}\left(\frac{q}{2N} i \quad mod \quad q\right)
$$

Necesitamos ahora computar $(\bigstar) = x^{-\tilde{\mu}^*} \cdot \overline{v}$, y para esto es que usamos la compuerta $\text{Cmux}$

Necesitamos tener los bits $s_i$ de la llave original $\vec{s}$, pero no se pueden dar en claro. Se proveen encriptados bajo otra llave $\vec{\bf{s}} \in \mathbb{B}_N[x]^k$, como $bsk[i] = \overline{\text{GLWE}}_{\vec{\bf{s}}}(s_i)$, y con el siguiente código podemos computar $(\bigstar)$:

$$
\begin{align*}
    &\text{ACC} \leftarrow (0, ... , 0 , x^{-\bar{b}}) \\
    &\text{for } i = 1,2,...,n:\\
    & \quad \text{ACC} \leftarrow \text{Cmux}(bsk[i], \text{ACC}, X^{\tilde{a}_i} \cdot \text{ACC})\\
    &\text{Devolvemos ACC}

\end{align*}
$$
El acumulador $\text{ACC}$ en el j-ésimo paso $\overline{c}_j = \text{ACC}_j$ guarda una encripción válida de $x^{-\tilde{b} + \sum_{i=1}^j \tilde{a}_i s_i} \cdot \overline{v}$ bajo $\vec{\bf{s}}$. Por tanto, al final, obtenemos una encripción
$$
\overline{\bf{c'}} = \overline{c}_n = \text{GLWE}_{\vec{\bf{s}}'}\left( x^{{-\tilde{b}} + \sum_{i=1}^n \tilde{a}_i {s}_i } \cdot \overline{v}\right) = \text{GLWE}_{\vec{\bf{s}}'}({\overline{u}})
$$


Ahora, $\overline{u} =  x^{{-\tilde{b}} + \sum_{i=1}^n \tilde{a}_i {s}_i } \cdot \overline{v} \in \hat{\mathbb{Z}}_N[x]$ es un polinomio cuyo término constante es $\overline{\mu}$ 

Realmente, si tenemos 
$$  
\vec{\bf{s}}'  = \begin{pmatrix} 
\bf{s}_1 \\
\vdots \\
\bf{s}_k
\end{pmatrix}  = \begin{pmatrix}
s_{10} + s_{11}x + \cdots + s_{1(N-1)}x^{N-1} \\
\vdots \\
s_{k0} + s_{k1}x + \cdots + s_{k(N-1)}x^{N-1}
\end{pmatrix} = \begin{pmatrix}
(s_{10}, \cdots, s_{1(N-1)}) \\
\vdots \\
(s_{k0}, \cdots, s_{k(N-1)})
\end{pmatrix} \\
\\

\\
\overline{\bf{c'}} \leftarrow \overline{\text{GLWE}}_{\vec{\bf{s}}'}(\overline{u}) =  ( \overline{\bf{a}}_1, ... , \overline{\bf{a}}_ k , \overline{\bf{b}}) \in \hat{\mathbb{Z}}_N [x] ^{k+1} 
=\begin{pmatrix}
a_{10} + a_{11}x + \cdots + a_{1(N-1)}x^{N-1} \\
\vdots \\
a_{k0} + a_{k1}x + \cdots + a_{k(N-1)}x^{N-1} \\
b_0 + b_1x + \cdots + b_{N-1}x^{N-1}
\end{pmatrix}
$$

Haciendo $ \overline{\bf{c}}^* = (a_{10}, -a_{1(N-1)}, \cdots, -a_{11}, a_{20}, \cdots, -a_{k0}, -a_{k(N-1)}, \cdots, -a_{k1}, b_0) \in \hat{\mathbb{Z}}[x]^{k+1 + kN} $, se puede verificar que corresponde a una salida aválida de $\overline{\text{LWE}}_{\bf{s}}^* ( \overline{\mu})$, con $\bf{s}^* = (s_{10}, s_{11}, \cdots, s_{kN}) \in \mathbb{B}^{kN}$

Tenemos entonces dos encripciones distintas para un mismo texto plano $\overline{\mu} \in \mathbb{Z} / q\mathbb{Z}$, bajo una llave original $\bf{s} \in \mathbb{B}^n$, y una llave originada del bootstrapping $\bf{s}^*$
$$ \overline{\bf{c}} \leftarrow \overline{\text{LWE}}_{\bf{s}} ( \overline{\mu}) \in (\mathbb{Z} / q \mathbb{Z})^{n+1}  \quad \text{Texto cifrado original} \quad \quad \\
\overline{\bf{c}}^{*} \leftarrow \overline{\text{LWE}}_{\bf{s}^*} ( \overline{\mu}) \in (\mathbb{Z} / q \mathbb{Z})^{kN+1}  \quad \text{Texto cifrado fresco} $$

El problema es que están en formatos distintos y con llaves distintas. Encriptando los bits de la llave original bajo $\bf{s}^*$, podemos hacer una operación llamada _key-switching_ para cambiar llaves entre conjuntos de parámetros diferentes, con una idea similar al bootstrapping. La diferencia principal con este, es que aumenta el ruido, pero es más sencillo de calcular. Con esto, podriamos usar el siguiente procedimiento para hacer bootstrapping:

<img src="misc/7.png" width="500" />

Sin embargo, el error producido por el key-switching amplifica el error de arrastre, y no se obtienen realmente reencripciones frescas. Pero si más bien, trabajamos con textos cifrados nativos en $(\mathbb{Z} / q \mathbb{Z})^{kN+1}$, podemos cambiar el procedimiento y obtener esto:

<img src="misc/8.png" width="500" />

El ruido generado por el key-switching se hace al principio, por lo que es más controlado, y se obtienen mejores resultados. Esto es lo que se usa para hacer bootstrapping

Ahora, podemos usar esta misma heurística para evaluar funciones arbitrarias! Si tenemos $f : \mathcal{D} \rightarrow \mathcal{I
}$, y funciones para codificar $\text{encode}_1, \text{encode}_2 : \mathcal{D},\mathcal{I} \rightarrow \mathbb{Z}/q\mathbb{Z}$ y decodificar $\text{decode}_1,\text{decode}_2 :\mathbb{Z}/q\mathbb{Z} \rightarrow  \mathcal{D},\mathcal{I} $, respectivamente, podemos usar la misma idea de un polinomio test.

Antes, queriamos que los coeficientes del polinomio almacenaran todos los valores "frescos" posibles para cualquier posible texto cifrado, y poder extraer después el valor que se necesitara. Nada impide hacer lo mismo en este caso. 

Hacemos una tabla con parejas $[i,T[i]]$ dada por $T[i] = \text{encode}_2 \circ f \circ \text{decode}_1 \circ \text{Upper}(\frac{q}{2N} i mod q)$ para $i = 0,1,...,N-1$. Guardamos esta tabla entonces como el polinomio $\overline{\bf{v}} = T[0]x^0 + T[1]x^1 + ... + T[N-1]$, y procedemos de la misma manera que en el bootstrapping. 

Fuentes:

- Dolev, S., Margalit, O., Pinkas, B., & Schwarzmann, A. (Eds.). (2021). Cyber Security Cryptography and Machine Learning. Lecture Notes in Computer Science. doi:10.1007/978-3-030-78086-9 

- Bourse, F., Minelli, M., Minihold, M., & Paillier, P. (2018). Fast Homomorphic Evaluation of Deep Discretized Neural Networks. Advances in Cryptology – CRYPTO 2018, 483–512. doi:10.1007/978-3-319-96878-0_17 

- Brakerski, Z., & Vaikuntanathan, V. (2014). Efficient Fully Homomorphic Encryption from (Standard) $\mathsf{LWE}$. SIAM Journal on Computing, 43(2), 831–871. doi:10.1137/120868669 

- Chillotti, I., Gama, N., Georgieva, M., & Izabachène, M. (2019). TFHE: Fast Fully Homomorphic Encryption Over the Torus. Journal of Cryptology. doi:10.1007/s00145-019-09319-x 